In [1]:
import pandas as pd

def rule_based_classifier(data, user_preferences):
    results = data.copy()

    # Filter by VitaminA and VitaminC
    results = results[(results['VitaminA'] == user_preferences['vitamin_a']) &(results['VitaminC'] == user_preferences['vitamin_c'])]

    # Filter by Fat, Cholesterol, Sodium, Carbohydrate, Protein, Calcium_mg, Sugar_g, Iron_mg, and Calories
    for key, value_range in user_preferences['ranges'].items():
        results = results[(results[key] >= value_range[0]) & (results[key] <= value_range[1])]
    # Filter by Contains
    if user_preferences['exclude_ingredients']:
        for ingredient in user_preferences['exclude_ingredients']:
            results = results[~results['Contains'].str.contains(ingredient)]
    # print(results)
    # Filter by Safe and Not_recommended
    results = results[(results['Safe'].str.contains(user_preferences['health_condition'])) | (~results['Not_recommended'].str.contains(user_preferences['health_condition']))]

    return results
import pandas as pd

# Load your dataset
data = pd.read_csv("final_products.csv")

# # Preprocess dataset
# data.dropna(inplace=True)
data['Contains'] = data['Contains'].str.lower()
data.head()

Product_ID         Name                          Safe Not_recommended  \
0         1.0   AdrenalAid         adrenal_health;stress            none   
1         2.0      AgeDefy        anti-aging;skin_health            none   
2         3.0  AlkaDefense        alkaline_balance;detox            none   
3         4.0  AquaHydrate        hydration;electrolytes            none   
4         5.0    BabyBelly  pregnancy_nausea;indigestion            none   

   VitaminA  VitaminC  Fat  Cholesterol  Sodium  Carbohydrate  Protein  \
0       0.0       1.0  3.0          5.0    15.0           3.0     30.0   
1       1.0       1.0  1.0          5.0    15.0           3.0     20.0   
2       0.0       0.0  2.0          5.0    20.0           4.0     15.0   
3       0.0       1.0  0.0          0.0     5.0           2.0      0.0   
4       0.0       1.0  5.0         10.0    30.0           2.0     45.0   

   Calcium_mg  Sugar_g  Iron_mg  Calories                          Contains  \
0       120.0      5.0      2.0      50.0      ashwagandha;rhodiola;ginseng   
1       150.0      7.0      1.0      55.0       resveratrol;coq10;vitamin_c   
2        80.0      6.0      2.0      40.0  spirulina;chlorella;barley_grass   
3         0.0      0.0      1.0      15.0           electrolytes;b-vitamins   
4       180.0      7.0      5.0      75.0      ginger;vitamin_b6;peppermint   

  Rating  N_reviews  
0    2.5       52.0  
1    4.8       93.0  
2    3.9       15.0  
3    3.4       72.0  
4    1.6       61.0

In [2]:
# User preferences
user_preferences = {
    'vitamin_a': 1,
    'vitamin_c': 1,
    'ranges': {
        'Fat': (1, 5),
        'Cholesterol': (5, 20),
        'Sodium': (5, 25),
        'Carbohydrate': (1, 40),
        'Protein': (20, 150),
        'Calcium_mg': (100, 200),
        'Sugar_g': (3, 7),
        'Iron_mg': (1, 4),
        'Calories': (40, 70)
    },
    'exclude_ingredients': ['Ginseng', 'Omega-3'],
    'health_condition': 'stress'
}

# Calculate the weighted score
rating_weight = 0.6
reviews_weight = 1 - rating_weight

# Make sure exclude_ingredients are in lowercase
user_preferences['exclude_ingredients'] = [ingredient.lower() for ingredient in user_preferences['exclude_ingredients']]
df=data
# Convert the 'Rating' and 'N_reviews' columns to numeric types
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
df['N_reviews'] = pd.to_numeric(df['N_reviews'], errors='coerce')

df['Weighted_Score'] = (df['Rating'] * rating_weight) + (df['N_reviews'] * reviews_weight / 100)

# Sort the DataFrame by the weighted score
df = df.sort_values(by='Weighted_Score', ascending=False)

# Call the rule_based_classifier function
recommended_products = rule_based_classifier(df, user_preferences)
recommended_products[:10]


Product_ID          Name                                Safe  \
1          2.0       AgeDefy              anti-aging;skin_health   
33        34.0    HairGrowth             hair_loss;brittle_nails   
7          8.0  BoneStrength  lactose_intolerance;kidney_disease   
51        52.0  MightyMarrow              osteoporosis;fractures   
12        13.0  CardioCharge         heart_health;blood_pressure   
30        31.0   GutGuardian                digestion;gut_health   
59        60.0  PrenatalPlus         pregnancy;fetal_development   
66        67.0   SkinRadiant              skin_health;complexion   
31        32.0     GutRelief                gut_health;digestion   
26        27.0    FlexiJoint                joint_pain;arthritis   

   Not_recommended  VitaminA  VitaminC  Fat  Cholesterol  Sodium  \
1             none       1.0       1.0  1.0          5.0    15.0   
33            none       1.0       1.0  2.0          5.0    20.0   
7             none       1.0       1.0  4.0          8.0    25.0   
51            none       1.0       1.0  4.0          8.0    25.0   
12    hypertension       1.0       1.0  2.0          6.0    20.0   
30            none       1.0       1.0  3.0          6.0    25.0   
59            none       1.0       1.0  4.0          8.0    25.0   
66            none       1.0       1.0  1.0          5.0    20.0   
31            none       1.0       1.0  3.0          8.0    20.0   
26            none       1.0       1.0  3.0          6.0    20.0   

    Carbohydrate  Protein  Calcium_mg  Sugar_g  Iron_mg  Calories  \
1            3.0     20.0       150.0      7.0      1.0      55.0   
33           1.0     30.0       150.0      5.0      3.0      55.0   
7            1.0     35.0       200.0      3.0      4.0      70.0   
51           1.0     35.0       200.0      3.0      4.0      70.0   
12           2.0     25.0       100.0      5.0      2.0      45.0   
30           3.0     40.0       180.0      6.0      4.0      70.0   
59           4.0     40.0       200.0      4.0      4.0      60.0   
66           2.0     20.0       100.0      7.0      1.0      45.0   
31           3.0     30.0       150.0      7.0      4.0      60.0   
26           2.0     30.0       150.0      6.0      4.0      55.0   

                             Contains  Rating  N_reviews  Weighted_Score  
1         resveratrol;coq10;vitamin_c     4.8       93.0           3.252  
33            biotin;keratin;collagen     4.8       60.0           3.120  
7       calcium;vitamin_d3;vitamin_k2     4.5       87.0           3.048  
51          magnesium;vitamin_d;boron     4.1       89.0           2.816  
12          beetroot;l-arginine;coq10     4.3        3.0           2.592  
30     probiotics;enzymes;l-glutamine     3.4       59.0           2.276  
59                folic_acid;iron;dha     2.3       92.0           1.748  
66          biotin;collagen;vitamin_e     1.6       78.0           1.272  
31  probiotics;prebiotics;l-glutamine     1.7       42.0           1.188  
26        glucosamine;chondroitin;msm     1.8       22.0           1.168

# 

In [3]:
user_preferences

{'vitamin_a': 1,
 'vitamin_c': 1,
 'ranges': {'Fat': (1, 5),
  'Cholesterol': (5, 20),
  'Sodium': (5, 25),
  'Carbohydrate': (1, 40),
  'Protein': (20, 150),
  'Calcium_mg': (100, 200),
  'Sugar_g': (3, 7),
  'Iron_mg': (1, 4),
  'Calories': (40, 70)},
 'exclude_ingredients': ['ginseng', 'omega-3'],
 'health_condition': 'stress'}